In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nb
import CMR_HFpEF_Analysis.functions_collection as ff

In [2]:
data_path = '/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/simulated_data'
patient_list = ff.sort_timeframe(ff.find_all_target_files(['*'],data_path),0)
print(len(patient_list))

1331


In [3]:
patient_id_list = []
for p in patient_list:
    i = ff.find_timeframe(p,0)
    tf_folders = ff.find_all_target_files(['ED'],p)
    # if len(tf_folders)!= 2: # have both ED and ES
    #     continue
    patient_id_list.append(i)
print(len(patient_id_list))

# 80% train, 20% test
import math
training_id = patient_id_list[0: 1030]
test_id = patient_id_list[1030: len(patient_id_list)]
print(len(training_id),len(test_id))

1331
1030 301


In [4]:
batch_size = int(len(training_id) / 5)
ds_num = 1
normal_num = 15


Result = []
for id_list in [training_id,test_id]:
    for i in range(0,len(id_list)):
        patient_id = id_list[i]
        if patient_id in training_id:
            train = 'train'
            batch = i // batch_size
        else:
            train = 'test'
            batch = 5
        
        simulate_folder = os.path.join(data_path,str(patient_id))
        simulate_tf_folder = ff.find_all_target_files(['ED'],simulate_folder)
        
        for tf_folder in simulate_tf_folder:
            tf = os.path.basename(tf_folder)

            # check lv slice condition
            condition = np.load(os.path.join(tf_folder,'lv_slice_condition.npy'),allow_pickle = True)
            remain_slice_num = len(condition[2][0])

            if remain_slice_num <= 6:
                continue

            # ground truth contours(HR)
            truth_file_HR = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/processed_HR_data',str(patient_id), tf, 'HR_ED_zoomed_crop_flip.nii.gz')

            # ground truth contours(HR) + cleaned contours
            truth_file_HR_clean = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/processed_HR_data',str(patient_id), tf, 'HR_ED_zoomed_crop_flip_clean.nii.gz')

            # ground truth downsampled contours(LR)
            truth_file_LR = os.path.join(tf_folder, 'ds/data.nii.gz')

            # ground truth downsampled + cleaned contours(LR)
            truth_file_LR_clean = os.path.join(tf_folder, 'ds/data_clean.nii.gz')

            # # assess whether it has 7 LV slices:
            # LR_clean_img = nb.load(truth_file_LR_clean).get_fdata(); LR_clean_img = np.round(LR_clean_img).astype(int)
            # nonempty_slices = len([z_slice for z_slice in range(LR_clean_img.shape[2]) if np.sum(LR_clean_img[:, :, z_slice]) != 0])
            # if nonempty_slices < 7:
            #     print(patient_id)
            
            # ground truth center points
            gt_center_points_file = os.path.join(tf_folder,'ds/centerlist.npy')
           
            r = [train,batch, patient_id, tf]
                
            # normal_motion
            for j in range(0,normal_num):
                input_file = os.path.join(tf_folder ,'normal_motion_' + str(j+1) ,'data.nii.gz')
                input_file_clean = os.path.join(tf_folder ,'normal_motion_' + str(j+1) ,'data_clean.nii.gz')
                motion_center_points_file = os.path.join(tf_folder ,'normal_motion_' + str(j+1) ,'centerlist.npy')
                motion_name = os.path.basename(os.path.dirname(input_file))
                Result.append(r + [motion_name, truth_file_HR, truth_file_HR_clean, truth_file_LR,  truth_file_LR_clean, gt_center_points_file, input_file, input_file_clean, motion_center_points_file])

In [9]:
df = pd.DataFrame(Result, columns = ['train','batch','Patient_ID','tf','motion_name','GroundTruthHR','GroundTruthHR_clean','GroundTruthLR','GroundTruthLR_clean','GroundTruth_CenterPoints','MotionFile', 'MotionFile_clean' , 'Motion_CenterPoints'])
df.to_excel(os.path.join('/mnt/mount_zc_NAS/HFpEF/data/Patient_list/Patient_list_train_test_simulated_data_15normals.xlsx'),index=False)

# For SunnyBrook Data

In [2]:
data_path = '/mnt/mount_zc_NAS/HFpEF/data/Sunny_Brooks/sunnybrooks_nii/'
patient_list = ff.find_all_target_files(['*'],data_path)
print(len(patient_list))

patient_id_list = []
for p in patient_list:
    i = os.path.basename(p)
    patient_id_list.append(i)

43


In [3]:
Result = []
for i in range(0,len(patient_id_list)):
    patient_id = patient_id_list[i]
    train = 'test'
    batch = 5
        
    patient_folder = os.path.join(data_path,patient_id)
    
    tf = ''

    # ground truth contours(HR)
    truth_file_HR = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/processed_HR_data','100/ED', 'HR_ED_zoomed_crop_flip.nii.gz')

    # ground truth contours(HR) + cleaned contours
    truth_file_HR_clean = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/processed_HR_data','100/ED', 'HR_ED_zoomed_crop_flip_clean.nii.gz')

    # ground truth downsampled contours(LR)
    truth_file_LR = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/simulated_data', '100/ED', 'ds/data.nii.gz')

    # ground truth downsampled + cleaned contours(LR)
    truth_file_LR_clean = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/simulated_data', '100/ED', 'ds/data_clean.nii.gz')

    # ground truth center points
    gt_center_points_file = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/simulated_data', '100/ED', 'ds/center_list.npy')
           
    r = [train,batch, patient_id, tf]
                
    # normal_motion
    input_file = ''
    input_file_clean = os.path.join(patient_folder, 'SAX_ED_for_DL.nii.gz' )
    motion_center_points_file = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/contour_dataset/simulated_data', '100/ED', 'ds/centerlist.npy')
    motion_name = ''
    Result.append(r + [motion_name, truth_file_HR, truth_file_HR_clean, truth_file_LR,  truth_file_LR_clean, gt_center_points_file, input_file, input_file_clean, motion_center_points_file])

In [4]:
df = pd.DataFrame(Result, columns = ['train','batch','Patient_ID','tf','motion_name','GroundTruthHR','GroundTruthHR_clean','GroundTruthLR','GroundTruthLR_clean','GroundTruth_CenterPoints','MotionFile', 'MotionFile_clean' , 'Motion_CenterPoints'])
df.to_excel(os.path.join('/mnt/mount_zc_NAS/HFpEF/data/Patient_list/Patient_list_SunnyBrooks.xlsx'),index=False)

# for HR data

In [ ]:
batch_size = int(len(training_id) / 5)
ds_num = 1
normal_num = 10


Result = []
for id_list in [training_id,test_id]:
    for i in range(0,len(id_list)):
        patient_id = id_list[i]
        if patient_id in training_id:
            train = 'train'
            batch = i // batch_size
        else:
            train = 'test'
            batch = 5
        
        simulate_folder = os.path.join(data_path,str(patient_id))
        simulate_tf_folder = ff.find_all_target_files(['*'],simulate_folder)
        
        for tf_folder in simulate_tf_folder:
            tf = os.path.basename(tf_folder)

            # check lv slice condition
            condition = np.load(os.path.join(tf_folder,'lv_slice_condition.npy'),allow_pickle = True)
            remain_slice_num = len(condition[2][0])

            if remain_slice_num <= 6:
                continue

            # ground truth (HR)
            truth_file_HR = os.path.join('/mnt/mount_zc_NAS/HFpEF/data/processed_HR_data',str(patient_id), tf, 'HR_'+tf+'_crop_60_flip.nii.gz')

            # ground truth (LR)
            truth_file_LR = os.path.join(tf_folder, 'ds/data_flip_clean.nii.gz')

            # ground truth (center points)
            center_points_file = os.path.join(tf_folder,'ds/HR/HR_centerlist_LV_slices.npy')
           
            r = [train,batch, patient_id, tf]
                

            # normal_motion
            for j in range(0,normal_num):
                pred_path = os.path.join('/mnt/mount_zc_NAS/HFpEF/predict/Combined/Iteration_C/round_1/images')
                input_file = os.path.join(pred_path, str(patient_id), tf ,'normal_motion_' + str(j+1) ,'pred_img_HR_final_flip_clean.nii.gz')
                motion_name = os.path.basename(os.path.dirname(input_file))
                Result.append(r + [motion_name, truth_file_HR, truth_file_LR, center_points_file, input_file])


            # severe_motion
            # for j in range(0,severe_2x_num):
            #     input_file = os.path.join(tf_folder ,'normal_motion_' + str(j+1) + '_2x' ,'data_flip_clean.nii.gz')
            #     motion_name = os.path.basename(os.path.dirname(input_file))
            #     Result.append(r + [motion_name, truth_file_HR, truth_file_LR, center_points_file, input_file])

            # for j in range(0,severe_2x_num):
            #     input_file = os.path.join(tf_folder ,'normal_motion_' + str(j+1) + '_2.5x' ,'data.nii.gz')
            #     motion_name = os.path.basename(os.path.dirname(input_file))
            #     Result.append(r + [motion_name, truth_file_HR, truth_file_LR, center_points_file, input_file])

            # for j in [4]:#range(0,severe_num):
            #     input_file = os.path.join(tf_folder ,'normal_motion_' + str(j+1) + '_3x' ,'data.nii.gz')
            #     motion_name = os.path.basename(os.path.dirname(input_file))
            #     Result.append(r + [motion_name, truth_file, input_file])



In [ ]:
df = pd.DataFrame(Result, columns = ['train','batch','Patient_ID','tf','motion_name','GroundTruthHR','GroundTruthLR','CenterPoints','InputFile'])
df.to_excel(os.path.join('/mnt/mount_zc_NAS/HFpEF/data/Patient_list/Patient_list_train_test_HR_LVslices_motion_flip_clean_7_slice_10_normal_IterationC.xlsx'),index=False)